# Imports

In [1]:
!pip install emoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import emoji
import string
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import spacy
import contractions
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
spacy_nlp = spacy.load('en_core_web_sm')

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!gdown 1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
!gdown 1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1

Downloading...
From: https://drive.google.com/uc?id=1OS2Aurb2oKmooCeYwYw0cnp542t7dyHo
To: /content/test_data.csv
100% 565k/565k [00:00<00:00, 72.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Oibb4sFzJNLt0iBWOD4LAIKaWT7cCNT1
To: /content/train_data.csv
100% 2.24M/2.24M [00:00<00:00, 76.9MB/s]


# Tokenizers

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [ ]:
X_train = train_df['clean_text'].to_numpy()
X_test = test_df['clean_text'].to_numpy()

In [ ]:
# Tokenize text data
tokenizer_any = Tokenizer(num_words=7000)
tokenizer_any.fit_on_texts(np.concatenate((X_train, X_test), axis=0))

In [ ]:
train_df_80 = train_df[train_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
test_df_80 = test_df[test_df['confidence'] >= 0.8].reset_index(drop=True)

In [ ]:
X_train_80 = train_df_80['clean_text'].to_numpy()
X_test_80 = test_df_80['clean_text'].to_numpy()

In [ ]:
# Tokenize text data
tokenizer_80 = Tokenizer(num_words=7000)
tokenizer_80.fit_on_texts(np.concatenate((X_train_80, X_test_80), axis=0))

# Models

In [ ]:
model_any = tf.keras.models.load_model("/content/drive/MyDrive/NLP/Stress Detection/Best_Models/Only Text/only_text_fasttext_any.keras")

In [ ]:
model_80 = tf.keras.models.load_model("/content/drive/MyDrive/NLP/Stress Detection/Best_Models/Only Text/only_text_fasttext_80.keras")

# Preprocessing Function

In [5]:
def html_remover(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

def url_remover(text):
    pattern = re.compile(r'https?.//\S+|www\.\S+')
    return pattern.sub(r'', text)

def number_remover(text):
  return re.sub(r"\w+\d+\w+|\w+\d+|\d+\w+|\d+", "", text)

stop = set(stopwords.words('english'))
def stopword_remover(words):
    return [word for word in words if word not in stop]

# ps = PorterStemmer()

def preprocessing(text):
  text = contractions.fix(text)
  text = str(text).lower()
  text = emoji.replace_emoji(text)
  text = html_remover(text)
  text = url_remover(text)
  text = number_remover(text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text)
  tokens = stopword_remover(tokens)
  doc = spacy_nlp(' '.join(tokens))
  return ' '.join(token.lemma_ for token in doc)
  # return ' '.join(ps.stem(word) for word in tokens)


In [10]:
preprocessing("Okay, So in since October have just got out of an eight year relationship . We were engaged to get married next year, but it did not work out. It ended mutually and we have moved on. In life these things happen. Since Halloween I have been seeing this Girl.")

'okay since october get eight year relationship engage get marry next year work end mutually move life thing happen since halloween see girl'

In [ ]:
def ready_text_any(text):
  text = preprocessing(text)
  text = tokenizer_any.texts_to_sequences(np.array([text]))
  text = pad_sequences(text, maxlen=150)
  return text
def ready_text_80(text):
  text = preprocessing(text)
  text = tokenizer_80.texts_to_sequences(np.array([text]))
  text = pad_sequences(text, maxlen=150)
  return text

# Predictions with Only Text model with any Confidence

In [7]:
stress_text1 = "Deadline looming, code crashing, client demanding revisions, coffee spilled, internet down, pressure mounting, errors multiplying, time slipping away, stress levels skyrocketing, chaos consuming, panic setting in, breathe!"
stress_text2 = "Why does everything always have to be so complicated? It feels like I'm drowning in a sea of never-ending tasks, deadlines, and expectations. Can't catch a break even for a moment. When will this relentless cycle of stress ever end?"
stress_text3 = "It feels like I'm constantly racing against time, trying to keep up with endless demands. Just when I think I'm catching my breath, another wave of responsibilities crashes down. It's exhausting, and I can't shake off this feeling of being overwhelmed."

In [ ]:
model_any.predict(ready_text_any(stress_text1))

1/1 [==============================] - 0s 22ms/step


array([[0.8691734]], dtype=float32)

In [ ]:
model_any.predict(ready_text_any(stress_text2))

1/1 [==============================] - 0s 28ms/step


array([[0.84948504]], dtype=float32)

In [ ]:
model_any.predict(ready_text_any(stress_text3))

1/1 [==============================] - 0s 23ms/step


array([[0.8691084]], dtype=float32)

In [ ]:
non_stress_text1 = "Today is such a beautiful day! The sun is shining, birds are singing, and everything just feels so peaceful. I'm grateful for moments like these that remind me of the simple joys in life. 😊"
non_stress_text2 = "Taking a moment to breathe and appreciate the little joys in life can really make a difference. Sometimes a simple walk outside or a warm cup of tea is all it takes to brighten the day."

In [ ]:
model_any.predict(ready_text_any(non_stress_text1))

1/1 [==============================] - 0s 20ms/step


array([[0.10368387]], dtype=float32)

In [ ]:
model_any.predict(ready_text_any(non_stress_text2))

1/1 [==============================] - 0s 24ms/step


array([[0.03688087]], dtype=float32)

# Predictions with Only Text model with 80% Confidence

In [ ]:
model_80.predict(ready_text_80(stress_text1))

1/1 [==============================] - 0s 36ms/step


array([[0.94947124]], dtype=float32)

In [ ]:
model_80.predict(ready_text_80(stress_text2))

1/1 [==============================] - 0s 20ms/step


array([[0.8977969]], dtype=float32)

In [ ]:
model_80.predict(ready_text_80(stress_text3))

1/1 [==============================] - 0s 22ms/step


array([[0.93939507]], dtype=float32)

In [ ]:
model_80.predict(ready_text_80(non_stress_text1))

1/1 [==============================] - 0s 21ms/step


array([[0.06730619]], dtype=float32)

In [ ]:
model_80.predict(ready_text_80(non_stress_text2))

1/1 [==============================] - 0s 37ms/step


array([[0.02528632]], dtype=float32)